$\bf{Problem Statement}$: Write a function that takes (ra,dec,distance,proper motion ra,proper motion dec,line-of-sight velocity) and their Gaussian uncertainties for a star and computes eccentricity, zmax, rperi, rap and their uncertainties using Monte Carlo sampling with the galpy function galpy.actionAngle.actionAngleStaeckel.EccZmaxRperiRap. Apply to some stars in the Gaia RV sample.

In [185]:
import numpy as np
from galpy.actionAngle import actionAngleStaeckel
from galpy.potential import MWPotential2014
from astropy.coordinates import ICRS, Galactocentric
import astropy.units as u

In [186]:
# create a action angle Staeckle object using the approximation delta = 0.4 and runs on c that makes the code faster
aAS = actionAngleStaeckel(pot=MWPotential2014,delta=0.4,c=True)

In [187]:
def ICRS_to_Galactocentric_Cylindrical(ra, dec, distance, pm_ra, pm_dec, vr):
    """
    Take a star's coordinate in ICRS form and return the galactocentric cylindrical coordinate in a tuple.
    
    Args:
        ra (float): the right ascension in degree
        dec (float): the declination in degree
        distance (float): the distance in kpc
        pm_ra (float): the proper motion in the right ascension direction in milliarcsec per year
        pm_dec (float): the proper motion in the declination direction in milliarcsec per year
        radial_velocity (float): the radial velocity in kilometer per second
        
    Returns:
        (R, phi, z, vR, vT, vz): a tuple of 6 Quantity representing the galactocentric cylindrical coordinates
    
    """
    # create an astroy coordinate object that stores the star's coordinate in ICRS form
    star = coord.ICRS(ra = ra*u.degree, dec = dec*u.degree, distance = distance*u.kpc, pm_ra_cosdec = pm_ra*u.mas/u.yr,
              pm_dec = pm_dec*u.mas/u.yr, radial_velocity = vr*u.km/u.s)
    # convert it to galactocentric coordinate
    star = star.transform_to(Galactocentric)
    # change it to cylindrical polar coordinate
    star.representation = 'cylindrical'
    # get the cylindrical polar coordinate attributes
    R, phi, z, vR, d_phi, vz  = star.rho, star.phi, star.z, star.d_rho, star.d_phi, star.d_z
    # change angular speed to unit of radian and take out the radian unit
    d_phi.to(u.rad/u.yr)
    d_phi = d_phi/u.rad
    # convert angular speed to tangential speed
    vT = d_phi * R
    # return the attributes
    return (R, phi, z, vR, vT, vz)

def find_EccZmaxRperiRap(ra, dec, distance, pm_ra, pm_dec, vr):
    # get the galactocentric cylindrical coordinate
    R, phi, z, vR, vT, vz  = ICRS_to_Galactocentric_Cylindrical(ra, dec, distance, pm_ra, pm_dec, vr)
    # compute the star's eccentricity, zmax, rperi, rap
    Ecc, Zmax, Rperi, Rap = aAS.EccZmaxRperiRap(R,vR,vT,z,vz,phi)
    return  (Ecc, Zmax, Rperi, Rap)


def EccZmaxRperiRap_with_uncertainties (ra, dec, distance, pm_ra, pm_dec, vr, ra_u, dec_u, distance_u, pm_ra_u, pm_dec_u, vr_u):
    """
    Args:
        ra (float): the right ascension in degree
        dec (float): the declination in degree
        distance (float): the distance in kpc
        pm_ra (float): the proper motion in the right ascension direction in milliarcsec per year
        pm_dec (float): the proper motion in the declination direction in milliarcsec per year
        radial_velocity (float): the radial velocity in kilometer per second
        
    Returns:
        Ecc (float): the eccentricity of the orbit 
        Zmax (float): the maximum height of the orbit
        Rperi (float): 
        Rap (float):
    
    """
    Ecc, Zmax, Rperi, Rap = find_EccZmaxRperiRap(ra, dec, distance, pm_ra, pm_dec, vr)
    
    
    


In [188]:
star = coord.ICRS(ra = 1*u.degree, dec = 1*u.degree, distance = 1*u.kpc, pm_ra_cosdec = 1*u.mas/u.yr,
              pm_dec = 1*u.mas/u.yr, radial_velocity = 1*u.km/u.s)

star = star.transform_to(Galactocentric)
star.representation = 'cylindrical'
R, phi, z, vR, vT, vz  = star.rho, star.phi, star.z, star.d_rho, star.d_phi, star.d_z
print(R, phi, z, vR, vT, vz)

8.39616064068874 kpc 176d35m28.28s -0.835639539061056 kpc 1.9674795419415028 kpc mas / (rad yr) -5.877872351137361 mas / yr 1.6400270194930917 kpc mas / (rad yr)
